Grid Search：网格搜索，一种调参手段；穷举搜索：在所有候选的参数选择中，通过循环遍历，尝试每一种可能性，表现最好的参数就是最终的结果。其原理就像是在数组里找最大值。

# 网格搜索

sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

- estimator: 模型
- param_grid: 要循环的参数，字典或含字典的列表
- scoring: 评估指标
- cv: 交叉验证折数

属性：

- best_estimator: 最优模型
- best_score_: 最高分数
- best_params_: 最优参数

## 未调参

In [1]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC

iris = load_iris()
svc = SVC()
svc.fit(iris.data, iris.target)
svc.score(iris.data, iris.target)

0.9733333333333334

## 调参后

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

iris = load_iris()

parameters = {'gamma': [0.001,0.01,0.1,1,10,100],
              'C': [0.001,0.01,0.1,1,10,100]}

svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)

# 最高分数
print(clf.best_score_)
# 最优模型
print(clf.best_estimator_)
# 最优参数
print(clf.best_params_)

0.9800000000000001
SVC(C=1, gamma=0.1)
{'C': 1, 'gamma': 0.1}


# 随机搜索

[参考](https://www.cnblogs.com/wj-1314/p/10422159.html)

我们在搜索超参数的时候，如果超参数个数较少（三四个或者更少），那么我们可以采用网格搜索，一种穷尽式的搜索方法。但是当超参数个数比较多的时候，我们仍然采用网格搜索，那么搜索所需时间将会指数级上升。

所以有人就提出了随机搜索的方法，随机在超参数空间中搜索几十几百个点，其中就有可能有比较小的值。这种做法比上面稀疏化网格的做法快，而且实验证明，随机搜索法结果比稀疏网格法稍好。

sklearn.model_selection.RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', random_state=None, error_score=nan, return_train_score=False)

- estimator: 模型
- param_distributions: 要循环的参数，字典或含字典的列表
- n_iter: 迭代次数
- scoring: 评估指标
- cv: 交叉验证折数
- random_state: 随机数

属性：

- best_estimator: 最优模型
- best_score_: 最高分数
- best_params_: 最优参数

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

iris = load_iris()

parameters = {'gamma': np.arange(0, 100, 0.1),
              'C': [0.001,0.01,0.1,1,10,100]}

svc = SVC()

clf = RandomizedSearchCV(svc, parameters, n_iter=300)

clf.fit(iris.data, iris.target)

print(clf.best_score_)
print(clf.best_params_)

0.9666666666666668
{'gamma': 4.1000000000000005, 'C': 1}


# 贝叶斯优化

参考资料：

[贝叶斯优化的Bayes优化参数调整原理与实践](https://www.pythonf.cn/read/136216)

[调参神器贝叶斯优化（bayesian-optimization）实战篇](https://www.jianshu.com/p/92d8943fb0ba)

In [43]:
# 先看看不调参之前的结果
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

x, y = make_classification(n_samples=1000,n_features=10,n_classes=2)
gbdt = GradientBoostingClassifier()
cross_val_score(gbdt, x, y, cv=5, scoring='roc_auc').mean()

## bayes调参初探

我们先定义一个目标函数，里面放入我们希望优化的函数。比如此时，函数输入为GradientBoostingClassifier的参数，输出为模型交叉验证5次的AUC均值，作为我们的目标函数。因为bayes_opt库只支持最大值，所以最后的输出如果是越小越好，那么需要在前面加上负号，以转为最大值。由于bayes优化只能优化连续超参数，因此要加上int()转为离散超参数。

BayesianOptimization().maximize(init_points=5, n_iter=25)

init_points,执行随机搜索的步数;n_iter,执行贝叶斯优化的步数

默认：init_points=5, n_iter=25

迭代次数由两部分组成，随机搜索的步数和贝叶斯优化的步数，贝叶斯优化的步数要多一点，步骤越多，就越有可能找到一个好的最大值。随机探索可以通过扩大探索空间而有所帮助。这里以迭代30次为例。

In [44]:
from bayes_opt import BayesianOptimization
def gbdt_cv(n_estimators, min_samples_split, max_features, max_depth):
    res = cross_val_score( 
        GradientBoostingClassifier(n_estimators=int(n_estimators),
                                min_samples_split=int(min_samples_split),
                                max_features=min(max_features, 0.999), # float
                                max_depth=int(max_depth),
                                random_state=2
        ),
        x, y, scoring='roc_auc', cv=5
    ).mean()
    return res


# 然后我们就可以实例化一个bayes优化对象了
# 里面的第一个参数是我们的优化目标函数，第二个参数是我们所需要输入的超参数名称，以及其范围。超参数名称必须和目标函数的输入名称一一对应。
gbdt_op = BayesianOptimization(
        gbdt_cv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999),
        'max_depth': (5, 15)}
    )
# 运行bayes优化
gbdt_op.maximize()

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.9684   |  11.66    |  0.1046   |  7.42     |  181.8    |
|  2        |  0.9697   |  6.005    |  0.911    |  11.27    |  75.46    |
|  3        |  0.9685   |  10.58    |  0.5531   |  4.687    |  193.1    |
|  4        |  0.9676   |  9.528    |  0.9797   |  10.95    |  100.9    |
|  5        |  0.9693   |  13.01    |  0.3497   |  9.929    |  146.8    |
|  6        |  0.9671   |  6.117    |  0.1382   |  10.64    |  75.7     |
|  7        |  0.9689   |  10.49    |  0.9917   |  6.654    |  216.8    |
|  8        |  0.9711   |  8.685    |  0.5938   |  24.69    |  106.5    |
|  9        |  0.9708   |  6.369    |  0.5485   |  10.02    |  57.18    |
|  10       |  0.9675   |  8.441    |  0.9778   |  10.76    |  36.36    |
|  11       |  0.9683   |  6.996    |  0.1021   |  13.38    |  164.4    |
|  12       |  0.9695   |  14.1     | 

In [45]:
# 查看当前最优的参数和结果
print(gbdt_op.max)

{'target': 0.9723799999999999, 'params': {'max_depth': 5.778950785247591, 'max_features': 0.4899642525170448, 'min_samples_split': 8.58901183740917, 'n_estimators': 43.31502231986158}}


In [62]:
# 用最优的参数组合定义一个新的GBDT，在和之前同样的数据上做训练，并生成评估得分。
gbdt1 = GradientBoostingClassifier(n_estimators=43,
                                   max_depth=5,
                                   min_samples_split=9,
                                   max_features=0.49,
                                   random_state=2)
# 调参后
print(cross_val_score(gbdt1, x, y, cv=5, scoring='roc_auc').mean())
# 调参前
print(cross_val_score(gbdt, x, y, cv=5, scoring='roc_auc').mean())

0.97227
0.97156
